In [9]:
import pandas as pd

# Cargar el archivo de Excel en un DataFrame
df = pd.read_excel(r"C:\Users\ArnaldoCarrasco\Documents\COTY\SO SCRIPT\Stock Farmacity\Stock Farmacity Nov-Marzo Consolidado.xlsx")

In [10]:
df_copy=df

In [11]:
df=df_copy

In [12]:
df = df.drop(['CUF', 'Proveedor', 'SubCategoria', 'Categoria', 'Tipo Rotacion', 'DEPTO', 'FC_STK_SEU', 'CD que Abastece', 'Estado Compra'], axis=1)
df = df.rename(columns={'Stock cierre periodo': 'Stock'})
df = df.rename(columns={'Descripcion': 'Desc Prod'})
df = df.rename(columns={'EAN': 'Ean'})
df = df.rename(columns={'Sucursal': 'Punto de Venta'})
df['Punto de Venta']=339
df['Grupo economico']= 60
df['Stock'] = df['Stock'].fillna(0)
df = df[df['Stock'] != 0]
df['Stock']=df['Stock'].astype(int)
df = df[df['Ean'] != 7795646617417]
df

,Fecha,Punto de Venta,Ean,Desc Prod,Stock,Grupo economico
1264,2024-12-01,339,3607345064536,POLVO COMPACTO STAY MATTE 04 X 45 GR,3,60
1270,2024-12-01,339,3607345064543,POLVO COMPACTO STAY MATTE 05 X 45 GR,3,60
1271,2024-12-01,339,3607345064550,POLVO COMPACTO STAY MATTE 06 X 45 GR,2,60
1277,2024-12-01,339,5012874125463,CORRECTOR OJERAS HIDE BLEMISH 05 X 16 GR,3,60
1283,2024-12-01,339,5012874101009,DELINEADOR LIQUIDO NEGRO FIBRA X 7 GR,4,60
...,...,...,...,...,...,...
599963,2025-03-01,339,3614226326195,MASCARA SCANDALEYES WOW MASCARA 001,5,60
599990,2025-03-01,339,30122505,MASCARA DE PESTANAS WONDERLUX NEGRO,1,60
599993,2025-03-01,339,8005610629612,LABIAL LIQUIDO LIPFINITY VELVET MATTE 015,3,60
599994,2025-03-01,339,8005610629698,LABIAL LIQUIDO LIPFINITY VELVET MATTE 025,2,60


In [13]:
file_path2 = r'C:\Users\ArnaldoCarrasco\Documents\COTY\SO SCRIPT\00.1 SO Enero\EAN_Mapeo.xlsx'
file_path3 = r'C:\Users\ArnaldoCarrasco\Documents\COTY\SO SCRIPT\00.0 Dimensionales\GPS.xlsx'
file_path4 = r'C:\Users\ArnaldoCarrasco\Documents\COTY\SO SCRIPT\00.0 Dimensionales\Buscador EAN.xlsx'
# Cargar el archivo de Excel en un DataFrame
ean_mapeo= pd.read_excel(file_path2)
gps= pd.read_excel(file_path3)
ean= pd.read_excel(file_path4)
gps_cliente= pd.read_excel(file_path3,sheet_name='CLIENTES')

In [14]:
ean_mapeo

,Ean,Marca,Desc Prod,Grupo economico Final,Ean Correcto,Tester
0,79170457629,SALLY HANSEN,SALLY H.ESM.PURE 110 WHITE TEA 7629 ...,MARIA UÑA DE CARLETTO,74170457629,NaN
1,3616306096622,NaN,HB/BOSS ABSOLU/WEEK/BOLSO OBS,PERFUGROUP,0,Si
2,3616303477134,NaN,HB/BOSS BOTT/ELIXIR BAG OBS,PERFUGROUP,0,Si
3,49151,NaN,ZZCOOL WATER GAME HOMBRE TESTE,PERFUGROUP,0,Si
4,7795646617417,NaN,BOM SALLY HANSEN MG CLUSTER ALTO MARZO 2025,FARMACITY,0,Si
...,...,...,...,...,...,...
691,2059300051005,NaN,Tester Risque Gwp Vanguarda 510,FARMALIFE S.A.,0,Si
692,2036138020502,NaN,Tester Sally Hansen Gwp Mg To The Taupe 3.0 205,FARMALIFE S.A.,0,Si
693,2036153413105,NaN,Tester Sally Hansen Gwp Pure Honey Harmony 131,FARMALIFE S.A.,0,Si
694,2036153435008,NaN,Tester Sally Hansen Gwp Pure Renov Soothing Sl...,FARMALIFE S.A.,0,Si


In [15]:
ean_historico = ean_mapeo[(ean_mapeo['Ean Correcto'] > 0) | ((ean_mapeo['Ean Correcto'] == 0) & (ean_mapeo['Tester'] == 'Si'))]
# ean_historico
df_corregido = pd.merge(
    df,
    ean_historico[['Desc Prod', 'Ean Correcto', 'Tester']],
    on='Desc Prod',
    how='left'
)
df_corregido = df_corregido[df_corregido['Tester'] != 'Si']
df_corregido.loc[df_corregido['Ean Correcto'] > 0, 'Ean'] = df_corregido.loc[df_corregido['Ean Correcto'] > 0, 'Ean Correcto']
df_corregido.drop(columns=['Ean Correcto', 'Tester'], inplace=True)

df=df_corregido

In [16]:
df["Tipo de Venta"] = "Sell out"
# agregar fechas de archivo y completar los nulos
# df["Fecha"]="1/1/2025"
# df.loc[df["Fecha"].isna(), "Fecha"] = pd.to_datetime("1/3/2025", format="%d/%m/%Y")
df["Fecha"] = pd.to_datetime(df["Fecha"], errors='coerce', dayfirst=True)
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d/%m/%Y")
df['Ean'] = df['Ean'].fillna(0)
df["Ean sist"] = df["Ean"].isin(ean["EAN"])

In [17]:
# Normalizar una columna específica (minúsculas y sin acentos)
def normalizar_columna(col):
    col.astype(str)
    return col.str.lower().apply(unidecode).str.strip()

df_merged = pd.merge(
    df,
    gps[["Código Coty","Codigo cliente","Cliente","Ecommerce"]],
    left_on=["Punto de Venta","Grupo economico"],
    right_on=["Codigo cliente","Cliente"],
    how='left',
    indicator=True
)

In [18]:
df_merged2 = pd.merge(
    df_merged,
    gps_cliente[["ID_CLIENTE","Nombre Correcto"]],
    left_on=["Grupo economico"],
    right_on=["ID_CLIENTE"],
    how='left'
    # indicator=True
)

In [19]:
# Lista de columnas a eliminar
columnas_a_eliminar = ["Cliente","ID_CLIENTE"]

# Eliminar columnas del DataFrame
df_merged2 = df_merged2.drop(columns=columnas_a_eliminar)
df=df_merged2

In [20]:
df = df.rename(columns={"Nombre Correcto": "Grupo economico Final"})
df = df.rename(columns={"Código Coty": "PDV GPS"})
df["Punto de Venta"] = df["Punto de Venta"].astype(str)
gps["Codigo cliente"] = gps["Codigo cliente"].astype(str)

In [21]:
df.loc[df['Grupo economico Final'] == 'DROGUERIA 20 DE JUNIO S.A.', ['PDV GPS', 'Ecommerce']] = ['DROGUERIA 20 DE JUNIO S.A.',False]
# df.loc[df['Grupo economico Final'] == 'DROGUERIA 20 DE JUNIO S.A.', 'PDV GPS'] = 'DROGUERIA 20 DE JUNIO S.A.'
df.loc[df['Grupo economico'] == 167, ['PDV GPS', 'Ecommerce']] = ['COFARSUR S.A.',False]
df.loc[df['Grupo economico Final'] == 'FIRST LABEL SRL', ['PDV GPS', 'Ecommerce']] = ['FIRST LABEL SRL.', True]
df.loc[df['Grupo economico Final'] == 'NIPPON', ['PDV GPS', 'Ecommerce']] = ['Nippon Nippon', False]
df.loc[df['Grupo economico Final'] == 'DROGUERIA SUR S.A.', ['PDV GPS', 'Ecommerce']] = ['DROGUERIA SUR S.A.', False]
df.loc[df['Grupo economico Final'] == 'DROGUERIA SUIZO', ['PDV GPS', 'Ecommerce']] = ['DROGUERIA SUIZO', False]
df.loc[df['Grupo economico Final'] == 'FARMAX SRL', ['PDV GPS', 'Ecommerce']] = ['FARMAX SRL', False]
df.loc[df['Grupo economico Final'] == 'TIENDA LOS GALLEGOS', ['PDV GPS', 'Ecommerce']] = ['TIENDA LOS GALLEGOS', False]
df.loc[df['Grupo economico Final'] == 'PRODEFARM S.A.', ['PDV GPS', 'Ecommerce']] = ['PRODEFARM S.A.', False]
df.loc[df['Grupo economico'] == 32, ['PDV GPS', 'Ecommerce']] = ['PERFUMERIAS ALDAZ', False]
df.loc[df['Grupo economico'] == 173, ['PDV GPS', 'Ecommerce']] = ['CASTRO JOSE MANUEL Y OTRO SOC.DE HECHO', False]
df.loc[df['Grupo economico'] == 166, ['PDV GPS', 'Ecommerce']] = ['KELLER', False]
df.loc[df['Grupo economico'] == 99, ['PDV GPS', 'Ecommerce']] = ['TIENDA LOS GALLEGOS', False]
df.loc[df['Grupo economico'] == 32, ['PDV GPS', 'Ecommerce']] = ['PREFUMERIAS ALDAZ', False]
df.loc[df['Grupo economico'] == 174, ['PDV GPS', 'Ecommerce']] = ['MAGLIONE', False]
df.loc[df['Grupo economico'] == 175, ['PDV GPS', 'Ecommerce']] = ['COOP.FARMACEUTICA MENDOZA LTDA.', False]
df.loc[df['Grupo economico'] == 176, ['PDV GPS', 'Ecommerce']] = ['ASOPROFARMA C.P.L.', False]
df.loc[df['Grupo economico'] == 177, ['PDV GPS', 'Ecommerce']] = ['ADM FRAGANCIAS S.R.L.', False]
df.loc[df['Grupo economico'] == 178, ['PDV GPS', 'Ecommerce']] = ['CASA DOAN', False]

In [22]:
df.loc[df['Grupo economico'] == 20, 'Punto de Venta'] = df.loc[df['Grupo economico'] == 20, 'Punto de Venta'].str.replace('.1', '')

C:\Users\ArnaldoCarrasco\AppData\Local\Temp\ipykernel_25380\246410927.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[df['Grupo economico'] == 20, 'Punto de Venta'] = df.loc[df['Grupo economico'] == 20, 'Punto de Venta'].str.replace('.1', '')


In [23]:
ean_errors = df[df['Ean sist'] == False]

In [24]:
ean_errors

,Fecha,Punto de Venta,Ean,Desc Prod,Stock,Grupo economico,Tipo de Venta,Ean sist,PDV GPS,Codigo cliente,Ecommerce,_merge,Grupo economico Final
168504,2025-03-01,339,7795646617417,BOM SALLY HANSEN MG CLUSTER ALTO MARZO 2025,15,60,Sell out,False,Farmacity: JUJUY 4 [Gral. Alvear 970],339,False,both,FARMACITY
168809,2025-03-01,339,7795646617417,BOM SALLY HANSEN MG CLUSTER ALTO MARZO 2025,24,60,Sell out,False,Farmacity: JUJUY 4 [Gral. Alvear 970],339,False,both,FARMACITY


In [25]:
# ean_errors
ean_errors_unique = ean_errors[["Ean", "Desc Prod", "Grupo economico Final"]].drop_duplicates().values.tolist()
ean_errors_unique = pd.DataFrame(ean_errors_unique, columns=["Ean", "Desc Prod", "Grupo economico Final"])
ean_errors_unique

,Ean,Desc Prod,Grupo economico Final
0,7795646617417,BOM SALLY HANSEN MG CLUSTER ALTO MARZO 2025,FARMACITY


In [26]:
gps_errors=df[df['_merge']=='left_only']
gps_errors_unique = gps_errors[["Grupo economico", "Grupo economico Final","Punto de Venta","Codigo cliente", "PDV GPS"]].drop_duplicates().values.tolist()
gps_errors_unique= pd.DataFrame(gps_errors_unique,columns=["Grupo economico", "Grupo economico Final","Punto de Venta","Codigo cliente", "PDV GPS"])
gps_errors_unique

,Grupo economico,Grupo economico Final,Punto de Venta,Codigo cliente,PDV GPS


In [27]:
ean_output_mapeo = pd.concat([ean_errors_unique, ean_historico], ignore_index=True)
ean_output_mapeo['Ean Correcto'] = ean_output_mapeo['Ean Correcto'].fillna(0)
ean_output_mapeo

,Ean,Desc Prod,Grupo economico Final,Marca,Ean Correcto,Tester
0,7795646617417,BOM SALLY HANSEN MG CLUSTER ALTO MARZO 2025,FARMACITY,NaN,0.000000e+00,NaN
1,79170457629,SALLY H.ESM.PURE 110 WHITE TEA 7629 ...,MARIA UÑA DE CARLETTO,SALLY HANSEN,7.417046e+10,NaN
2,3616306096622,HB/BOSS ABSOLU/WEEK/BOLSO OBS,PERFUGROUP,NaN,0.000000e+00,Si
3,3616303477134,HB/BOSS BOTT/ELIXIR BAG OBS,PERFUGROUP,NaN,0.000000e+00,Si
4,49151,ZZCOOL WATER GAME HOMBRE TESTE,PERFUGROUP,NaN,0.000000e+00,Si
...,...,...,...,...,...,...
692,2059300051005,Tester Risque Gwp Vanguarda 510,FARMALIFE S.A.,NaN,0.000000e+00,Si
693,2036138020502,Tester Sally Hansen Gwp Mg To The Taupe 3.0 205,FARMALIFE S.A.,NaN,0.000000e+00,Si
694,2036153413105,Tester Sally Hansen Gwp Pure Honey Harmony 131,FARMALIFE S.A.,NaN,0.000000e+00,Si
695,2036153435008,Tester Sally Hansen Gwp Pure Renov Soothing Sl...,FARMALIFE S.A.,NaN,0.000000e+00,Si


In [28]:
df_conteo = ean_output_mapeo.groupby("Grupo economico Final").size().reset_index(name="Ean por mapear")
# df_conteo

ean_output_mapeo=ean_errors_unique
ean_output_mapeo["Ean Correcto"]=0

ean_output_mapeo = pd.concat([ean_output_mapeo, ean_historico])
ean_output_mapeo

,Ean,Desc Prod,Grupo economico Final,Ean Correcto,Marca,Tester
0,7795646617417,BOM SALLY HANSEN MG CLUSTER ALTO MARZO 2025,FARMACITY,0,NaN,NaN
0,79170457629,SALLY H.ESM.PURE 110 WHITE TEA 7629 ...,MARIA UÑA DE CARLETTO,74170457629,SALLY HANSEN,NaN
1,3616306096622,HB/BOSS ABSOLU/WEEK/BOLSO OBS,PERFUGROUP,0,NaN,Si
2,3616303477134,HB/BOSS BOTT/ELIXIR BAG OBS,PERFUGROUP,0,NaN,Si
3,49151,ZZCOOL WATER GAME HOMBRE TESTE,PERFUGROUP,0,NaN,Si
...,...,...,...,...,...,...
691,2059300051005,Tester Risque Gwp Vanguarda 510,FARMALIFE S.A.,0,NaN,Si
692,2036138020502,Tester Sally Hansen Gwp Mg To The Taupe 3.0 205,FARMALIFE S.A.,0,NaN,Si
693,2036153413105,Tester Sally Hansen Gwp Pure Honey Harmony 131,FARMALIFE S.A.,0,NaN,Si
694,2036153435008,Tester Sally Hansen Gwp Pure Renov Soothing Sl...,FARMALIFE S.A.,0,NaN,Si


In [29]:
file_path = r'C:\Users\ArnaldoCarrasco\Documents\COTY\SO SCRIPT\Arvhivos de salida\EAN_Mapeo.xlsx'
ean_output_mapeo.to_excel(file_path, index=False)

In [30]:
grupo_ok=df

In [31]:
grupo_ok = grupo_ok.drop(columns=["Desc Prod", "Grupo economico", "Punto de Venta", "Ean sist","Codigo cliente", "_merge"])

In [32]:
# grupo_ok
grupo_ok.rename(columns={
    "PDV GPS": "Punto de Venta",
    "Grupo economico Final": "Grupo economico",
    "Ecommerce": "Columna1"
}, inplace=True)
grupo_ok["Columna1"] = grupo_ok["Columna1"].replace({True: "Si", False: "No"})
# grupo_ok

In [33]:
grupo_ok['Columna1']=""

In [34]:
grupo_ok

,Fecha,Ean,Stock,Tipo de Venta,Punto de Venta,Columna1,Grupo economico
0,2024-12-01,3607345064536,3,Sell out,Farmacity: JUJUY 4 [Gral. Alvear 970],,FARMACITY
1,2024-12-01,3607345064543,3,Sell out,Farmacity: JUJUY 4 [Gral. Alvear 970],,FARMACITY
2,2024-12-01,3607345064550,2,Sell out,Farmacity: JUJUY 4 [Gral. Alvear 970],,FARMACITY
3,2024-12-01,5012874125463,3,Sell out,Farmacity: JUJUY 4 [Gral. Alvear 970],,FARMACITY
4,2024-12-01,5012874101009,4,Sell out,Farmacity: JUJUY 4 [Gral. Alvear 970],,FARMACITY
...,...,...,...,...,...,...,...
200524,2025-03-01,3614226326195,5,Sell out,Farmacity: JUJUY 4 [Gral. Alvear 970],,FARMACITY
200525,2025-03-01,30122505,1,Sell out,Farmacity: JUJUY 4 [Gral. Alvear 970],,FARMACITY
200526,2025-03-01,8005610629612,3,Sell out,Farmacity: JUJUY 4 [Gral. Alvear 970],,FARMACITY
200527,2025-03-01,8005610629698,2,Sell out,Farmacity: JUJUY 4 [Gral. Alvear 970],,FARMACITY


In [35]:
file_path = r'C:\Users\ArnaldoCarrasco\Documents\COTY\SO SCRIPT\Stock Farmacity\salida\StockFarmacity.csv'
grupo_ok.to_csv(file_path, index=False)


In [36]:
# so_file_path= r"C:\Users\ArnaldoCarrasco\Documents\COTY\SO SCRIPT\Arvhivos de salida\MarzoFinal.csv"
# somarzo= pd.read_csv(so_file_path)

In [37]:
# result = pd.concat([grupo_ok, somarzo])

In [38]:
# file_path = r'C:\Users\ArnaldoCarrasco\Documents\COTY\SO SCRIPT\Stock Farmacity\salida\MarzoFinal.csv'
# result.to_csv(file_path, index=False)